<a href="https://colab.research.google.com/github/xuanhuyid/BigData/blob/DEEEP-LEARNING/3_label_ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3_label_ResNet101**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Lib

In [ ]:
import os
import cv2
import numpy as np
import PIL
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import keras
from tensorflow.keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
import tensorflow as tf
import os 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix,f1_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

## Data

### Create data path


In [ ]:

# Gán đường dẫn dữ liệu chính
path_data = '/content/data'

# Gán đường dẫn dữ liệu nén zip
path_data_zip = path_data + '/zip'
path_data_zip_train = path_data_zip + '/train.zip'
path_data_zip_test = path_data_zip + '/test.zip'

# Gán đường dẫn dữ liệu train, validation, test
path_data_train = path_data + '/train'
path_data_test = path_data + '/test'

# Kiểm tra đường dẫn dữ liệu chính có hay không và tạo mới nếu không có
if not os.path.isdir(path_data):
  os.makedirs(path_data)

### Download and unzip data

In [ ]:
# Kiểm tra nếu chưa có đường dẫn chứa data zip thì tải về
# tao bo data train
# test https://drive.google.com/file/d/15dolnTSvbk7QeBch4-h_VmBdwtMScu2X/view?usp=share_link
# train https://drive.google.com/file/d/1Hh07-y4fNZLBeLcDelCaxb_Z5jD1qrBk/view?usp=share_link
if not os.path.isdir(path_data_zip):
  os.makedirs(path_data_zip)
  !gdown --o $path_data_zip_train --id "1Hh07-y4fNZLBeLcDelCaxb_Z5jD1qrBk"
  !gdown --o $path_data_zip_test --id "15dolnTSvbk7QeBch4-h_VmBdwtMScu2X"
if not os.path.isdir(path_data_train):
  !unzip $path_data_zip_train -d $path_data
if not os.path.isdir(path_data_zip_test):
  !unzip $path_data_zip_test -d $path_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1414).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1415).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1416).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1417).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1418).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1419).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1420).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1421).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1423).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1424).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1425).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1426).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1427).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (1428).jpg  
  inflating: /content/data/train/CHUA D

### Data augmentation

In [ ]:
train_data_gen = ImageDataGenerator(rescale = 1./255.)
test_data_gen = ImageDataGenerator( rescale = 1.0/255. )
image_shape = (224, 224)
batch_size=32

In [ ]:
train_data=train_data_gen.flow_from_directory(path_data_train,target_size=image_shape,color_mode='rgb',batch_size=batch_size,shuffle=True,seed = 42)
test_data=test_data_gen.flow_from_directory(path_data_test,target_size=image_shape,color_mode='rgb',batch_size=batch_size,shuffle=False)

Found 8978 images belonging to 3 classes.
Found 949 images belonging to 3 classes.


In [ ]:
len(train_data.class_indices)

3

### Split y_test

In [ ]:
# arr_0=np.zeros(len(os.listdir(path_data_test+'/fire'),)).reshape(-1,1)
# print(arr_0.shape)
# arr_1=np.ones(len(os.listdir(path_data_test+'/nonfire'),)).reshape(-1,1)
# print(arr_1.shape)
# y_test=np.vstack((arr_0,arr_1))
# print(y_test.shape)
# #print(y_test)

# Model

## ResNet101


In [ ]:
base_model = ResNet101(input_shape = (224, 224, 3), 
include_top = False,
weights = 'imagenet')
print(base_model.summary())
for layer in base_model.layers:
    layer.trainable = False

171446536/171446536 [==============================] - 1s 0us/step
Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                       

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

x= layers.Dense(256, activation='relu')(x)

x= layers.Dropout(0.5)(x) 


# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(len(train_data.class_indices), activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

print(model.summary())

model.compile(optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5), loss = 'categorical_crossentropy',metrics = ['acc'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

### Train model

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/best_model.h5',
                                                      monitor='val_acc',
                                                      save_best_only=True)
ResNet101_model = model.fit(train_data, validation_data = test_data, steps_per_epoch = 50, epochs = 20,callbacks=[checkpoint_callback])


Epoch 1/20
50/50 [==============================] - 111s 2s/step - loss: 1.2462 - acc: 0.4056 - val_loss: 0.9994 - val_acc: 0.4910
Epoch 2/20
50/50 [==============================] - 92s 2s/step - loss: 1.0859 - acc: 0.4401 - val_loss: 0.9748 - val_acc: 0.5195
Epoch 3/20
50/50 [==============================] - 89s 2s/step - loss: 1.0287 - acc: 0.4968 - val_loss: 0.9585 - val_acc: 0.5648
Epoch 4/20
50/50 [==============================] - 82s 2s/step - loss: 1.0151 - acc: 0.4931 - val_loss: 0.9737 - val_acc: 0.5227
Epoch 5/20
50/50 [==============================] - 89s 2s/step - loss: 1.0039 - acc: 0.5150 - val_loss: 0.9452 - val_acc: 0.5827
Epoch 6/20
50/50 [==============================] - 84s 2s/step - loss: 0.9977 - acc: 0.5194 - val_loss: 0.9389 - val_acc: 0.5743
Epoch 7/20
50/50 [==============================] - 89s 2s/step - loss: 0.9763 - acc: 0.5344 - val_loss: 0.9323 - val_acc: 0.5848
Epoch 8/20
50/50 [==============================] - 91s 2s/step - loss: 0.9991 - acc: 0.5

In [ ]:
bath  = len(test_data)
pre = []
true = []
for i in range(bath):
  y_pred = model.predict(test_data[i][0])
  for j in y_pred:
    pre.append(np.argmax(j,axis = 0))
  for j in test_data[i][1]:
    true.append(np.argmax(j,axis = 0))
from sklearn.metrics import accuracy_score,f1_score
print(accuracy_score(true,pre),f1_score(true,pre,average = 'macro'),f1_score(true,pre,average = 'weighted'))

1/1 [==============================] - 2s 2s/step
0.5879873551106428 0.4219726970199354 0.5277288442995067


In [ ]:
model.load_weights('/content/drive/MyDrive/best_model.h5')
bath  = len(test_data)
pre = []
true = []
for i in range(bath):
  y_pred = model.predict(test_data[i][0])
  for j in y_pred:
    pre.append(np.argmax(j,axis = 0))
  for j in test_data[i][1]:
    true.append(np.argmax(j,axis = 0))
from sklearn.metrics import accuracy_score,f1_score
print(accuracy_score(true,pre),f1_score(true,pre,average = 'macro'),f1_score(true,pre,average = 'weighted'))

1/1 [==============================] - 0s 32ms/step
0.5964172813487882 0.4319190360737639 0.5394772939416526


### Save model

In [ ]:
model1.save('/content/drive/MyDrive/save/vgg16_1.h5')